# Compare HATVP declarations using Agents

## install libs

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers #install from source to get Agents
!pip install accelerate # needed to use Agents
!pip install datasets # needed to use Audio in PythonInterpreter
!pip install tiktoken

## load dataset

In [ ]:
from datasets import load_dataset
embed_ds = load_dataset("the-french-artist/hatvp_declarations_text_embeds", split='train')

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
small_test_df = embed_ds.select(range(10)).to_pandas()
small_test_df

,xml_sha1,declaration_xml,declaration_json,extracted_text,text_embedding
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",Fiche de damien abad - député/ain(01) \n -----...,"[-0.009466919116675854, -0.008166174404323101,..."
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",Fiche de damien abad - député/ain(01) \n -----...,"[-0.010201191529631615, 0.0008864270057529211,..."
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",Fiche de caroline abadie - député/isère(38) \n...,"[0.0018713556928560138, 0.00048493893700651824..."
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",Fiche de caroline abadie - député/isère(38) \n...,"[0.0008672993280924857, -0.003786470042541623,..."
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",Fiche de joelle abadie - elu départemental/hau...,"[-0.0008490124018862844, -0.01616096682846546,..."
5,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/09/2021 10:41:48</d...,"{""declaration"": {""dateDepot"": ""03/09/2021 10:4...",Fiche de muriel abadie - elu régional/occitani...,"[-0.019427385181188583, -0.017342902719974518,..."
6,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>21/08/2021 19:39:06</d...,"{""declaration"": {""dateDepot"": ""21/08/2021 19:3...",Fiche de brigitte abaux - autre \n -----------...,"[-0.011065250262618065, 0.0016156501369550824,..."
7,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>23/08/2021 21:37:05</d...,"{""declaration"": {""dateDepot"": ""23/08/2021 21:3...",Fiche de gérard abbas - elu départemental/meus...,"[0.0031800672877579927, 0.021794293075799942, ..."
8,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/08/2021 00:09:05</d...,"{""declaration"": {""dateDepot"": ""17/08/2021 00:0...",Fiche de véronique abelin-drapron - elu départ...,"[-0.014365713112056255, -0.007720843888819218,..."
9,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>28/09/2020 21:30:36</d...,"{""declaration"": {""dateDepot"": ""28/09/2020 21:3...",Fiche de jean-pierre abelin - maire ou adjoint...,"[0.007507301867008209, -0.019156564027071, -0...."


## load Agents -- CANCELLED: Agents are stupid. -- Load simple inference API completion

In [ ]:
from google.colab import userdata
from huggingface_hub import InferenceClient
from huggingface_hub import login

login(userdata.get('HF_TOKEN'))

def ask_question(prompt):
  messages = [{"role": "user", "content": prompt}]
  client = InferenceClient("meta-llama/Meta-Llama-3-8B-Instruct")
  response = client.chat_completion(messages, max_tokens=10)
  return response.choices[0].message.content

ask_question('What is the capital of France?')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


'The capital of France is Paris.'

In [ ]:
# from google.colab import userdata
# from huggingface_hub import login, InferenceClient
# from transformers import ReactCodeAgent, HfEngine

# login(userdata.get('HF_TOKEN'))

# llm_engine = HfEngine(model="meta-llama/Meta-Llama-3-8B-Instruct")
# agent = ReactCodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)

In [ ]:
# from transformers import CodeAgent

# agent = CodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)
# agent.run(
#     """
# What s the capital of France?
# """
# )

## Select two declaration texts and compare them using an Agent

### select two declarations

In [ ]:
declaration_1 = small_test_df.declaration_json.to_list()[1]
declaration_2 = small_test_df.declaration_json.to_list()[7]

In [ ]:
import json
print(json.dumps(json.loads(declaration_1), indent=2))

{
  "declaration": {
    "dateDepot": "27/11/2022 18:18:23",
    "uuid": "fa8d18ec-0db9-4a39-b1f4-caba0c31329b",
    "origine": "ADEL",
    "complete": "true",
    "attachedFiles": {
      "attachedFiles": {
        "fileName": "VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML",
        "serverFileName": null,
        "base64EncodedContent": ""
      }
    },
    "declarationVersion": "20171221",
    "activConsultantDto": {
      "neant": "true"
    },
    "activProfCinqDerniereDto": {
      "items": {
        "items": {
          "motif": {
            "id": "CREATION",
            "label": null
          },
          "commentaire": "Fonction exerc\u00e9e du 20 Mai au 4 Juillet 2022         [Donn\u00e9es non publi\u00e9es]",
          "description": "Ministre des Solidarit\u00e9s, de l'Autonomie et des Personnes handicap\u00e9es",
          "employeur": "MINISTERE DES SOLIDARITES, DE L'AUTONOMIE ET DES PERSONNES HANDICAPEES",
          "remuneration": {
            "brutNet": "Net",
            "mon

In [ ]:
print(json.dumps(json.loads(declaration_2), indent=2))

{
  "declaration": {
    "dateDepot": "23/08/2021 21:37:05",
    "uuid": "a2893249-f253-4cb4-bb9d-f7fcec066d31",
    "origine": "ADEL",
    "complete": "true",
    "attachedFiles": {
      "attachedFiles": {
        "fileName": "VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML",
        "serverFileName": null,
        "base64EncodedContent": ""
      }
    },
    "declarationVersion": "20171221",
    "activConsultantDto": {
      "neant": "true"
    },
    "activProfCinqDerniereDto": {
      "neant": "true"
    },
    "activProfConjointDto": {
      "neant": "true"
    },
    "fonctionBenevoleDto": {
      "neant": "true"
    },
    "mandatElectifDto": {
      "items": {
        "items": [
          {
            "motif": {
              "id": "CREATION",
              "label": null
            },
            "commentaire": "net per\u00e7u avant pr\u00e9l\u00e8vement ou imp\u00f4ts2021: \u00e0 fin juin",
            "descriptionMandat": "Maire de Fains-V\u00e9el",
            "remuneration": {
      

### check token count  

We use openai tiktoken, it seems that GPT and LLAMA tokens are +/- 5-10 appart in count.

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str = 'cl100k_base') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
num_tokens_from_string(declaration_1)

3465

In [ ]:
num_tokens_from_string(declaration_2)

2791

We check that the addition of these two test declarations is below the total context size of 8192

In [ ]:
num_tokens_from_string(declaration_1) + num_tokens_from_string(declaration_2)

6256

### Make the prompt

In [ ]:
prompt = f"""
You are an OSINT expert.
You are presented with two HATVP declarations.
Please find the similarities between the following two JSON declarations.
Do not analyze keys, only leaf values.
Do not report on common properties.
If there are no discernable similarities, state 'no similarities'.
Write code: True

Declaration 1:
{declaration_1}

Declaration 2:
{declaration_2}
"""


prompt = f"""
### instructions
You are an OSINT expert.
You are tasked with finding conflicts of interest.
You are presented with two HATVP declarations from two different declarants.
Please indicate interesting relationships between the two declarations.

### Declaration 1:

{declaration_1}

### Declaration 2:

{declaration_2}
"""


prompt = f"""
### instructions
You are an OSINT expert.
You are tasked with finding conflicts of interest.
You are presented with two HATVP declarations from two different declarants.
Please give a level of conflict of interest risk: 'LOW' 'MEDIUM' 'HIGH'.
Only write 'LOW' or 'MEDIUM' or 'HIGH'.

### Declaration 1:

{declaration_1}

### Declaration 2:

{declaration_2}
"""


prompt = f"""
### instructions
You are an OSINT expert.
You are tasked with finding conflicts of interest.
You are presented with a HATVP declaration.
Please give a level of conflict of interest risk: 'LOW' 'MEDIUM' 'HIGH'.
Only write 'LOW' or 'MEDIUM' or 'HIGH'.

### Declaration

{declaration_1}

"""

prompt = f"""
### instructions
You are an OSINT expert.
You are tasked with finding conflicts of interest.
You are presented with a HATVP declaration.
Please give a level of conflict of interest risk: 'LOW' 'MEDIUM' 'HIGH'.
Only write 'LOW' or 'MEDIUM' or 'HIGH'.

### Declaration

{declaration_2}

ONLY WRITE 'LOW' or 'MEDIUM' or 'HIGH'.

"""

We check again, and the complete prompt is under the 8k limit

In [ ]:
num_tokens_from_string(prompt)

2877

In [ ]:
response = ask_question(prompt)
print(response)

MEDIUM


In [ ]:
def get_risk_level_from_declaration(declaration_text):
  prompt = f"""
### instructions
You are an OSINT expert.
You are tasked with finding conflicts of interest.
You are presented with a HATVP declaration.
Please give a level of conflict of interest risk: 'LOW' 'MEDIUM' 'HIGH'.
Only write 'LOW' or 'MEDIUM' or 'HIGH'.

### Declaration

{declaration_text}

ONLY WRITE 'LOW' or 'MEDIUM' or 'HIGH'.

"""
  return ask_question(prompt)

In [ ]:
small_test_df['risk_level'] = small_test_df.declaration_json.apply(get_risk_level_from_declaration)

In [ ]:
small_test_df

,xml_sha1,declaration_xml,declaration_json,extracted_text,text_embedding,risk_level
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",Fiche de damien abad - député/ain(01) \n -----...,"[-0.009466919116675854, -0.008166174404323101,...",MEDIUM
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",Fiche de damien abad - député/ain(01) \n -----...,"[-0.010201191529631615, 0.0008864270057529211,...",MEDIUM
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",Fiche de caroline abadie - député/isère(38) \n...,"[0.0018713556928560138, 0.00048493893700651824...",MEDIUM
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",Fiche de caroline abadie - député/isère(38) \n...,"[0.0008672993280924857, -0.003786470042541623,...",HIGH
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",Fiche de joelle abadie - elu départemental/hau...,"[-0.0008490124018862844, -0.01616096682846546,...",MEDIUM
5,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/09/2021 10:41:48</d...,"{""declaration"": {""dateDepot"": ""03/09/2021 10:4...",Fiche de muriel abadie - elu régional/occitani...,"[-0.019427385181188583, -0.017342902719974518,...",MEDIUM
6,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>21/08/2021 19:39:06</d...,"{""declaration"": {""dateDepot"": ""21/08/2021 19:3...",Fiche de brigitte abaux - autre \n -----------...,"[-0.011065250262618065, 0.0016156501369550824,...",MEDIUM
7,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>23/08/2021 21:37:05</d...,"{""declaration"": {""dateDepot"": ""23/08/2021 21:3...",Fiche de gérard abbas - elu départemental/meus...,"[0.0031800672877579927, 0.021794293075799942, ...",MEDIUM
8,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/08/2021 00:09:05</d...,"{""declaration"": {""dateDepot"": ""17/08/2021 00:0...",Fiche de véronique abelin-drapron - elu départ...,"[-0.014365713112056255, -0.007720843888819218,...",MEDIUM
9,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>28/09/2020 21:30:36</d...,"{""declaration"": {""dateDepot"": ""28/09/2020 21:3...",Fiche de jean-pierre abelin - maire ou adjoint...,"[0.007507301867008209, -0.019156564027071, -0....",MEDIUM


In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "register_risk_level",
            "description": "Registers the risk level",
            "parameters": {
                "type": "object",
                "properties": {
                    "risk_level": {
                        "type": "string",
                        "enum": ["LOW", "MEDIUM", "HIGH"],
                        "description": "The conflict of interest risk level of a declaration",
                    },
                    "risk_reason": {
                        "type": "string",
                        "description": "The justification behind the choice of a given risk_level.",
                    },
                },
                "required": ["risk_level", "risk_reason"],
            },
        },
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "register_risk_level",
            "description": "Registers the risk level",
            "parameters": {
                "type": "object",
                "properties": {
                    "risk_level": {
                        "type": "string",
                        "enum": ["LOW", "MEDIUM", "HIGH"],
                        "description": "The conflict of interest risk level of a declaration",
                    },
                    "risk_reason": {
                      "type": "array",
                      "items": {
                        "type": "string",
                        "enum": [
                          "several-public-offices",
                          "high-ranking-official",
                          "unpaid-position",
                          "financial-assets",
                          "investment-in-companies",
                          "high-value-assets",
                          "consulting-activities",
                          "professional-activities",
                          "spouse-employment",
                          "family-financial-interests",
                          "volunteer-roles",
                          "non-profit-positions",
                          "real-estate",
                          "gifts-and-hospitality"
                        ]
                      }
                    }
                },
                "required": ["risk_level", "risk_reason"],
            },
        },
    }
]

def ask_question(prompt):
  messages = [{"role": "user", "content": prompt}]
  client = InferenceClient("meta-llama/Meta-Llama-3-8B-Instruct")
  response = client.chat_completion(
      messages=messages,
      tools=tools,
      tool_choice="auto",
      max_tokens=200)
  return response.choices[0].message.tool_calls[0].function.arguments

def get_risk_level_from_declaration(declaration_text):
  prompt = f"""
### instructions
You are an OSINT expert.
You are tasked with finding conflicts of interest.
You are presented with a HATVP declaration.
Please give a level of conflict of interest risk: 'LOW' 'MEDIUM' 'HIGH'.
Use function register_risk_level to respond.
Indicate risk reason in a few words.

### Declaration

{declaration_text}

"""
  return ask_question(prompt)

In [ ]:
response = get_risk_level_from_declaration(declaration_1)

In [ ]:
response['risk_level']

'MEDIUM'

In [ ]:
response['risk_reason']

['several-public-offices', 'professional-activities', 'financial-assets']

In [ ]:
import pandas as pd

def map_get_risk_level_from_declaration(declaration_text):
    response = get_risk_level_from_declaration(declaration_text)
    return response['risk_level'], response['risk_reason']

# Apply the function and expand the results into separate columns
small_test_df[['risk_level', 'risk_reason']] = small_test_df['declaration_json'].apply(map_get_risk_level_from_declaration).apply(pd.Series)

In [ ]:
small_test_df

,xml_sha1,declaration_xml,declaration_json,extracted_text,text_embedding,risk_level,risk_reason
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",Fiche de damien abad - député/ain(01) \n -----...,"[-0.009466919116675854, -0.008166174404323101,...",MEDIUM,"[several-public-offices, high-ranking-official..."
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",Fiche de damien abad - député/ain(01) \n -----...,"[-0.010201191529631615, 0.0008864270057529211,...",MEDIUM,"[several-public-offices, professional-activiti..."
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",Fiche de caroline abadie - député/isère(38) \n...,"[0.0018713556928560138, 0.00048493893700651824...",MEDIUM,"[several-public-offices, family-financial-inte..."
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",Fiche de caroline abadie - député/isère(38) \n...,"[0.0008672993280924857, -0.003786470042541623,...",HIGH,[spouse-employment]
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",Fiche de joelle abadie - elu départemental/hau...,"[-0.0008490124018862844, -0.01616096682846546,...",HIGH,"[several-public-offices, high-ranking-official]"
5,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/09/2021 10:41:48</d...,"{""declaration"": {""dateDepot"": ""03/09/2021 10:4...",Fiche de muriel abadie - elu régional/occitani...,"[-0.019427385181188583, -0.017342902719974518,...",MEDIUM,"[several-public-offices, high-ranking-official]"
6,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>21/08/2021 19:39:06</d...,"{""declaration"": {""dateDepot"": ""21/08/2021 19:3...",Fiche de brigitte abaux - autre \n -----------...,"[-0.011065250262618065, 0.0016156501369550824,...",MEDIUM,"[several-public-offices, financial-assets, inv..."
7,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>23/08/2021 21:37:05</d...,"{""declaration"": {""dateDepot"": ""23/08/2021 21:3...",Fiche de gérard abbas - elu départemental/meus...,"[0.0031800672877579927, 0.021794293075799942, ...",HIGH,"[several-public-offices, high-ranking-official..."
8,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/08/2021 00:09:05</d...,"{""declaration"": {""dateDepot"": ""17/08/2021 00:0...",Fiche de véronique abelin-drapron - elu départ...,"[-0.014365713112056255, -0.007720843888819218,...",HIGH,"[high-ranking-official, several-public-offices]"
9,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>28/09/2020 21:30:36</d...,"{""declaration"": {""dateDepot"": ""28/09/2020 21:3...",Fiche de jean-pierre abelin - maire ou adjoint...,"[0.007507301867008209, -0.019156564027071, -0....",LOW,"[non-profit-positions, non-profit-positions, n..."


In [ ]:
print(json.dumps(json.loads(small_test_df[small_test_df.risk_level == 'HIGH'].declaration_json.to_list()[0]), indent=2))

{
  "declaration": {
    "dateDepot": "04/10/2022 17:22:07",
    "uuid": "c40ab214-b21e-43b6-95a2-36d5860dc526",
    "origine": "ADEL",
    "complete": "true",
    "attachedFiles": {
      "attachedFiles": {
        "fileName": "VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML",
        "serverFileName": null,
        "base64EncodedContent": ""
      }
    },
    "declarationVersion": "20171221",
    "activConsultantDto": {
      "items": null,
      "neant": "true"
    },
    "activProfCinqDerniereDto": {
      "items": null,
      "neant": "true"
    },
    "activProfConjointDto": {
      "items": {
        "items": {
          "motif": {
            "id": "CREATION",
            "label": null
          },
          "commentaire": null,
          "nomConjoint": "[Donn\u00e9es non publi\u00e9es]",
          "employeurConjoint": "ODIGO",
          "activiteProf": "DIRECTEUR DE PROJET CADRE"
        }
      },
      "neant": "false"
    },
    "fonctionBenevoleDto": {
      "items": null,
      "neant

## TODO and CURRENT STATE:  
### CURRENT STATE
1. we dropped the agents, a simple API inference call is enough  
2. we cannot relate the risk level to the tags used for now, we might have to test a bigger sample to see if "LOW" risk declarations are found  
3. the "spouse-employment" doesn't seem relevant, we still have to check the other tags.  

### TODO
1. apply risk assessment on 100 samples  
2. expand the tag list to add non-threatning possibilities  
3. find a way to relate the tag choice to the actual content of a declaration  
4. maybe expand the risk level to a scale of 1 to 5, or 1 to 10 to get more granularity  
5. statistical analysis of risk level should be made (non random repartition, frequency, check that same individuals have the same risk level in their multiple declarations)  
6. find actual risky declarations and check their risk level  

### TODO2  

1. Find possible families in the dataset (parents, couples, etc...) based on name and birthdate  
2. Find common investments and common associations? (maybe not an LLM task...)  

